In [3]:
from ML_PROJECT.constants import *

In [5]:
from ML_PROJECT.utils.common import read_yaml,create_directories 

In [22]:
import os
os.getcwd()

'd:\\venkat\\MLOPS\\MLOPS End to End Project\\ML_Project_RedWine\\ML_Project_RedWine'

In [21]:
os.chdir('..')

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
from ML_PROJECT import logger
from ML_PROJECT.utils.common import get_size
from ML_PROJECT.entity.config_entity import DataIngestionConfig
from pathlib import Path

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
x=DataIngestion(DataIngestionConfig)

In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-02-10 23:27:09,496: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-10 23:27:09,498: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-10 23:27:09,501: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-10 23:27:09,503: INFO: common: created directory at: artifacts]
[2024-02-10 23:27:09,506: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-10 23:27:32,119: INFO: 3019190260: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9D96:3D74:1F07C6:2519FA:65C7B90A
Accept-Ranges: bytes
Date: Sat, 10 F